# Fig. S08-S13

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
import src.utils as utils
import src.ci_utils as ci_utils

palette0 = sns.color_palette(['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000']) # Okabe-Ito
palette = palette0
display(palette)
sns.set_palette(palette)
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

In [ ]:
value_type = 'mean'
save_dir = "../output/figure-for-paper/"

## Fig. S08 | Slope plot (5, 10, 15, 20 sec)

In [ ]:
def plot_slope_plot_pre_post(data_type_list, duration_sec_list, value_type='mean'):
    GRIDSPEC_KW = {'wspace': 0.5, 'hspace': 0.5} 
    fig, axes = plt.subplots(3, 4, figsize=(20, 12), gridspec_kw=GRIDSPEC_KW)
    
    for i, data_type in enumerate(data_type_list): 
        _, _, _, title_base = ci_utils.setup_for_causal_impact_analysis(data_type=data_type)
        if i == 0:
            # y_ticks = np.arange(-0.2, 2, 0.2)
            y_ticks = np.arange(-0.5, 2, 0.5)
            ymin, ymax = -0.05, 1.83
        elif i == 1:
            if data_type == 'speed_distance_km_h':
                y_ticks = np.arange(-0, 100, 30)
                ymin, ymax = -5, 95
            elif data_type == 'abs_diff_distance_m':
                y_ticks = np.arange(-2, 6, 1)
                ymin, ymax = -0.4, 3.8
        elif i == 2:
            if data_type == 'pixel_count_p':
                y_ticks = np.arange(0, 1, 0.1)
                ymin, ymax = -0.04, 0.21
            elif data_type == 'abs_diff_pixel_count_p':
                y_ticks = np.arange(0, 1, 0.01)
                ymin, ymax = -0.005, 0.033
        for j, duration_sec in enumerate(duration_sec_list):
            ax = axes[i, j]
            df, df2 = ci_utils.prep_df_pre_post_analysis(data_type, duration_sec, value_type)
            ax = ci_utils.vis_slope_plot_ax(df, df2, ax)
            ax.set_yticks(y_ticks)
            ax.set_ylim(ymin, ymax)
            ax.set_ylabel("")
            ax.yaxis.set_minor_locator(tck.AutoMinorLocator(2))
            ax.grid(which='minor', axis='y', alpha=0.3)
            ax.set_xlim(-0.3, 1.3)
            if i == 0:
                ax.set_title(f'{duration_sec} (s)', fontsize=24, pad=20)
            if j % len(duration_sec_list) == 0:
                # ax.set_ylabel(f'{data_y_label_list[i]} (s)', fontsize=24, labelpad=20)
                ax.annotate(f'{title_base} ', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                    xycoords=ax.yaxis.label, textcoords='offset points',
                    ha='right', va='center', fontsize=24, rotation=90)
    plt.show()
    return fig

In [ ]:
duration_sec_list = [5, 10, 15, 20]
data_type_list = ['smoothed_VeDBA_2s', 'abs_diff_distance_m', 'abs_diff_pixel_count_p']
fig = plot_slope_plot_pre_post(data_type_list, duration_sec_list)
fig.savefig(f"{save_dir}/png/fig_s08_diff_pre_post_slope.png", dpi=150, bbox_inches="tight", pad_inches=0.5, transparent=False)
fig.savefig(f"{save_dir}/pdf/fig_s08_diff_pre_post_slope.pdf", dpi=600, bbox_inches="tight", pad_inches=0.5, transparent=False)
plt.close()

In [ ]:
# show 1 plot
SEC = 5
data_type_list = ['abs_diff_pixel_count_p'] 
for data_type in data_type_list:
    GRIDSPEC_KW = {'wspace': 0.5, 'hspace': 0.5} 
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), gridspec_kw=GRIDSPEC_KW)
    df, df2 = ci_utils.prep_df_pre_post_analysis(data_type, SEC, value_type)
    ax = ci_utils.vis_slope_plot_ax(df, df2, ax)
    y_ticks = np.arange(0, 1, 0.05)
    ymin, ymax = -0.02, 0.06
    ax.set_yticks(y_ticks)
    ax.set_ylim(ymin, ymax)
    _, _, _, ylabel = ci_utils.setup_for_causal_impact_analysis(data_type)
    ax.set_ylabel(ylabel)
    ax.yaxis.set_minor_locator(tck.AutoMinorLocator(2))
    ax.grid(which='minor', axis='y', alpha=0.3)
    ax.set_xlim(-0.3, 1.3)
    plt.show()
    display(df.head(3))
    display(df2.head(3))

## Fig. S09 | Total effect (5, 10, 15, 20 sec)
The average intervention effects of all playback sessions (for predator and noise, respectively) can be formulated below.
  
$$E = \frac{1}{N} \sum_{i=0}^{N} ({Y_{1i}} - {Y_{0i}})$$

${Y_{1i}}$ is the post-mean value of one of the three response indicators for playback session $i$.  
${Y_{0i}}$ is the pre-mean value of one of the three response indicators for playback session $i$.  

In [ ]:
def plot_total_effect_by_playback_type(data_type_list, duration_sec_list):
    GRIDSPEC_KW = {'wspace': 0.5, 'hspace': 0.5} 
    fig, axes = plt.subplots(3, 4, figsize=(20, 12), gridspec_kw=GRIDSPEC_KW)
    for i, data_type in enumerate(data_type_list): 
        _, _, _, title_base = ci_utils.setup_for_causal_impact_analysis(data_type=data_type)
        if i == 0:
            y_ticks = np.arange(-1.2, 1.4, 0.4)
            ymin, ymax = -0.55, 0.55
        elif i == 1:
            if data_type == 'speed_distance_km_h':
                y_ticks = np.arange(-0, 100, 30)
                ymin, ymax = -5, 95
            elif data_type == 'abs_diff_distance_m':
                # y_ticks = np.arange(-2, 6, 1)
                # ymin, ymax = -0.4, 3.8
                y_ticks = np.arange(-2, 5, 2.0)
                ymin, ymax = -2.5, 3.5
        elif i == 2:
            if data_type == 'pixel_count_p':
                y_ticks = np.arange(-5.0, 5.0, 0.1)
                # ymin, ymax = -0.22, 0.22
                ymin, ymax = -0.22, 0.12
            elif data_type == 'abs_diff_pixel_count_p':
                y_ticks = np.arange(-1.04, 1.0, 0.02)
                ymin, ymax = -0.045, 0.045
        for j, duration_sec in enumerate(duration_sec_list):
            ax = axes[i, j]
            df, df2 = ci_utils.prep_df_pre_post_analysis(data_type, duration_sec, value_type)
            _df = df[df['audio_file_name'] != 'Cancelled']
            # Stripplot
            ax = sns.stripplot(x='audio_file_name', y='diff_value', hue='audio_file_name', data=_df, ax=ax)
            # Mean
            categories = _df["audio_file_name"].unique()
            means = _df.groupby("audio_file_name")["diff_value"].mean()
            for c, category in enumerate(categories):
                ax.hlines(y=means[category], xmin=c - 0.3, xmax=c + 0.3, color=palette0[c], linewidth=3)
            ax.set_yticks(y_ticks)
            ax.set_ylim(ymin, ymax)
            ax.yaxis.set_minor_locator(tck.AutoMinorLocator(2))
            ax.yaxis.set_major_formatter(tck.FuncFormatter(utils.custom_formatter_2f))
            ax.set_ylabel("")
            ax.set_xlabel("")
            ax.set_xlim(-0.5, 1.5)
            ax.grid(which='major', axis='y')
            ax.grid(which='minor', axis='y', alpha=0.3)
            
            if i == 0:
                ax.set_title(f'{duration_sec} (s)', fontsize=24, pad=20)
            if j % len(duration_sec_list) == 0:
                # ax.set_ylabel(f'{data_y_label_list[i]} (s)', fontsize=24, labelpad=20)
                ax.annotate(f'{title_base} ', xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                    xycoords=ax.yaxis.label, textcoords='offset points',
                    ha='right', va='center', fontsize=24, rotation=90)
    plt.show()
    return fig

In [ ]:
duration_sec_list = [5, 10, 15, 20]
data_type_list = ['smoothed_VeDBA_2s', 'abs_diff_distance_m', 'abs_diff_pixel_count_p']
fig = plot_total_effect_by_playback_type(data_type_list, duration_sec_list)
fig.savefig(f"{save_dir}/png/fig_s09_diff_pre_post_mean_by_audio.png", dpi=150, bbox_inches="tight", pad_inches=0.5, transparent=False)
fig.savefig(f"{save_dir}/pdf/fig_s09_diff_pre_post_mean_by_audio.pdf", dpi=600, bbox_inches="tight", pad_inches=0.5, transparent=False)
plt.close()

## Fig. S10-13 | Number of playbacks

In [ ]:
def plot_line_plot_by_playback_type(data_type_list, duration_sec_list):
    fig_list = []
    for s, sec in enumerate(duration_sec_list):
        GRIDSPEC_KW = {'width_ratios': [1, 1], 'wspace': 0.4, 'hspace': 0.6}
        fig, axes = plt.subplots(3, 2, figsize=(18, 14), gridspec_kw=GRIDSPEC_KW)
        ax_list = list(axes.flatten())
        
        for i, data_type in enumerate(data_type_list):
            _, _, _, title_base = ci_utils.setup_for_causal_impact_analysis(data_type=data_type)
            ax = ax_list[i]
            df, df2 = ci_utils.prep_df_pre_post_analysis(data_type, duration_sec=sec, value_type='mean') 
            
            if i % 2 == 0:
                audio_file = "Predator"
            else:
                audio_file = "Noise"
            ax, scatter = ci_utils.vis_line_plot_pre_post_diff_2_ax(ax, df, data_type, title_base, audio_file)

            if i == 0:
                handles, labels = scatter.get_legend_handles_labels()
                handles_list = handles
                labels_list = labels
            if i < 4:
                ax.set_xlabel("")
            else:
                ax.set_xlabel('Playback count', labelpad=10)

            # Label for plot
            plot_label_idx_list = [0, 2, 4]
            plot_label_list = ['(a)', '(b)', '(c)']
            for j, plot_label_idx in enumerate(plot_label_idx_list):
                if i == plot_label_idx:
                    ax.text(
                        -0.27, 1.1, f'{plot_label_list[j]}', transform=ax.transAxes, fontsize=30, fontweight='bold'
                    )
            
        # Show the common figure legend outside the axes
        fig.legend(handles_list, labels_list, loc='center', bbox_to_anchor=(1.0, 0.5), ncol=1)
        # fig.savefig(f"{save_dir}/fig_diff_pre_post_n_of_playback_{sec:02}_sec.svg", bbox_inches="tight", pad_inches=0.5, transparent=False)

        # if sec == 5:
        #     fig_number = "fig_s10":
        
        fig.savefig(f"{save_dir}/png/fig_s{s+10:02d}_diff_pre_post_n_of_playback_{sec:02d}_sec.png", dpi=150, bbox_inches="tight", pad_inches=0.5, transparent=False)
        fig.savefig(f"{save_dir}/pdf/fig_s{s+10:02d}_diff_pre_post_n_of_playback_{sec:02d}_sec.pdf", dpi=350, bbox_inches="tight", pad_inches=0.5, transparent=False)

        fig_list.append(fig)
        plt.show()
        plt.close()
        
    return fig_list

In [ ]:
# duration_sec_list = [5]
duration_sec_list = [5, 10, 15, 20]
data_type_list = [
    'smoothed_VeDBA_2s', 'smoothed_VeDBA_2s', 
    'abs_diff_distance_m', 'abs_diff_distance_m', 
    'abs_diff_pixel_count_p', 'abs_diff_pixel_count_p'
]
fig_list = plot_line_plot_by_playback_type(data_type_list, duration_sec_list)